# Carga Data TERMINOS X ROL

Creación del DataFrame asociado a los Terminos por Rol

Rev: 29-10-2020

In [1]:
import os
import pandas as pd
import numpy as np
from pyarrow import feather
from tqdm import tqdm

from pjud.data import cleandata

In [2]:
path_raw = "../data/raw/pjud" 
archivos = os.listdir(path_raw) 
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
# Creacion dataframe con los datos de Términos por Rol Penal de los años 2015 a 2019

dataframes = []

for archivo in archivos:
    if archivo.find("Términos por Rol Penal") != -1:
        df = pd.read_csv(f"{path_raw}/{archivo}", sep = ";", encoding = 'cp850', low_memory = True)
        dataframes.append(df) 
        
df_termino_rol = pd.concat(dataframes, axis = 0)

In [5]:
df_termino_rol.columns

Index(['N°', 'CÓD. CORTE', 'CORTE', 'CÓD. TRIBUNAL', 'TRIBUNAL', 'RIT',
       'TIPO CAUSA', 'FECHA INGRESO', 'FECHA TÉRMINO', 'DURACIÓN CAUSA ',
       'MOTIVO DE TÉRMINO', 'MES TÉRMINO', 'AÑO TÉRMINO', 'TOTAL TÉRMINOS',
       'SISTEMA'],
      dtype='object')

In [6]:
df_termino_rol['SISTEMA'].unique()

array(['SIAGJ', 'METGE', nan], dtype=object)

Elimino causas que no sean SIAGJ

In [7]:
df_no_siagj = df_termino_rol[df_termino_rol['SISTEMA']!='SIAGJ']

In [8]:
df_termino_rol.drop(df_no_siagj.index, axis=0, inplace=True)

In [9]:
# Elimino filas vacias o con datos NaN

df_termino_rol = df_termino_rol.dropna()
df_termino_rol.drop(['N°','SISTEMA'], axis = 'columns', inplace = True)

In [10]:
# Cambio de nombre a algunas columnas para dejarlas iguales a otros dataframes

df_termino_rol.rename(columns = {'CÓD. CORTE':'COD. CORTE',
                                'CÓD. TRIBUNAL':'COD. TRIBUNAL',
                                'DURACIÓN CAUSA ':'DURACION CAUSA',
                                'MOTIVO DE TÉRMINO':'MOTIVO TERMINO',
                                'FECHA TÉRMINO':'FECHA TERMINO',
                                'MES TÉRMINO':'MES TERMINO',
                                'AÑO TÉRMINO':'AÑO TERMINO',
                                'TOTAL TÉRMINOS':'TOTAL TERMINOS'
                                },
                     inplace = True) 

In [11]:
# Transformamos variables float64 a int16

df_termino_rol['COD. CORTE'] = df_termino_rol['COD. CORTE'].fillna(0).astype(np.int16)
df_termino_rol['COD. TRIBUNAL'] = df_termino_rol['COD. TRIBUNAL'].fillna(0).astype(np.int16)
df_termino_rol['DURACION CAUSA'] = df_termino_rol['DURACION CAUSA'].fillna(0).astype(np.int16)
df_termino_rol['AÑO TERMINO'] = df_termino_rol['AÑO TERMINO'].fillna(0).astype(np.int16)
df_termino_rol['TOTAL TERMINOS'] = df_termino_rol['TOTAL TERMINOS'].fillna(0).astype(np.int8)

In [12]:
# Elimino tildes de las columnas object

cols = df_termino_rol.select_dtypes(include = ["object"]).columns
df_termino_rol[cols] = df_termino_rol[cols].progress_apply(cleandata.elimina_tilde)

100%|██████████| 8/8 [00:59<00:00,  7.47s/it]


In [13]:
# Convertir objetos a fechas
#cols_date = ['FECHA INGRESO', 'FECHA TERMINO']
#df_termino_rol[cols_date] = df_termino_rol[cols_date].progress_apply(convierte_fecha)

df_termino_rol['FECHA INGRESO'] = df_termino_rol['FECHA INGRESO'].progress_apply(cleandata.convierte_fecha)
df_termino_rol['FECHA TERMINO'] = df_termino_rol['FECHA TERMINO'].progress_apply(cleandata.convierte_fecha)

100%|██████████| 3356831/3356831 [00:06<00:00, 555079.67it/s]


In [14]:
# Elimino espacios en las columnas tipo objetos

df_termino_rol = df_termino_rol.progress_apply(cleandata.elimina_espacios, axis=0)

100%|██████████| 13/13 [00:07<00:00,  1.78it/s]


In [15]:
df_termino_rol['RIT'] = df_termino_rol['RIT'].progress_apply(cleandata.limpia_rit)

100%|██████████| 3356831/3356831 [00:02<00:00, 1163652.73it/s]


In [16]:
# Transformamos en variables categoricas

df_termino_rol['CORTE'] = df_termino_rol['CORTE'].astype('category')
df_termino_rol['MOTIVO TERMINO'] = df_termino_rol['MOTIVO TERMINO'].astype('category')

Dejo solo causas Ordinarias

In [17]:
df_termino_rol['TIPO CAUSA'].unique()

array(['Ordinaria', 'Militar', 'Extradición'], dtype=object)

In [18]:
tipo_causa = df_termino_rol[df_termino_rol['TIPO CAUSA']!='Ordinaria']

In [19]:
df_termino_rol.drop(tipo_causa.index, axis=0, inplace=True)

In [20]:
df_termino_rol

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS
0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,2018-04-02,2018-06-01,60,SENTENCIA,jun-18,2018,1
1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,101-2018,Ordinaria,2018-04-02,2018-08-29,149,SENTENCIA,ago-18,2018,1
2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,10-2018,Ordinaria,2018-01-10,2018-03-14,63,SENTENCIA,mar-18,2018,1
3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,102-2018,Ordinaria,2018-04-02,2018-05-29,57,SENTENCIA,may-18,2018,1
4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,103-2018,Ordinaria,2018-04-02,2018-05-30,58,SENTENCIA,may-18,2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
679953,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,86-2014,Ordinaria,2014-12-02,2015-01-31,60,SENTENCIA,ene-15,2015,1
679954,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,87-2014,Ordinaria,2014-12-09,2015-01-28,50,SENTENCIA,ene-15,2015,1
679955,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,88-2014,Ordinaria,2014-12-30,2015-02-02,34,SENTENCIA,feb-15,2015,1
679956,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,90-2013,Ordinaria,2013-12-18,2015-08-17,607,SENTENCIA,ago-15,2015,1


In [21]:
# Reset el index para realizar feather
df_termino_rol.reset_index(inplace = True)

In [22]:
# Guardamos dataset como archivo feather
path_interim = "../data/interim/pjud"
os.makedirs(path_interim, exist_ok = True) 

df_termino_rol.to_feather(f'{path_interim}/clean_TerminosRol.feather')
